In [54]:
import requests
from bs4 import BeautifulSoup
import re

In [3]:
session = requests.session()

In [4]:
headers = {
    'authority': 'www.kith.com',
    'cache-control': 'max-age=0',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.106 Safari/537.36',
    'sec-fetch-dest': 'document',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-user': '?1',
    'accept-language': 'en-US,en;q=0.9',
}

## Вытащил последнюю страницу

In [19]:
start_url = "https://www.vpoxod.ru/route?per-page=48&page=500"
start_response = session.get(start_url, headers=headers)

In [20]:
start_response.status_code

200

In [21]:
start_soup = BeautifulSoup(start_response.text, 'html.parser')

In [35]:
last_page_num = int(start_soup.find(attrs={'class':'pagination'}).find(attrs={'class':'active disabled'}).text)

## Работаем со страницами

In [75]:
route_url = "https://www.vpoxod.ru"

In [37]:
page_url = "https://www.vpoxod.ru/route?per-page=48&page=1"
page_response = session.get(page_url, headers=headers)

In [38]:
page_response.status_code

200

In [40]:
page_soup = BeautifulSoup(page_response.text,'html.parser')

In [42]:
hike_list = page_soup.find_all(attrs={'class':'main_page_hike_title'})

In [76]:
hike_list_urls = [(route_url+route.find()['href']) for route in hike_list]

In [77]:
hike_list_urls

['https://www.vpoxod.ru/route/karelia/shuya#content-top',
 'https://www.vpoxod.ru/route/karelia/keret#content-top',
 'https://www.vpoxod.ru/route/karelia/pistaioki_splav#content-top',
 'https://www.vpoxod.ru/route/kolskiy/umba#content-top',
 'https://www.vpoxod.ru/route/siberia/putorana_komfort#content-top',
 'https://www.vpoxod.ru/route/kolskiy/tumcha#content-top',
 'https://www.vpoxod.ru/route/nw/Viborg_monrepo_kayak#content-top',
 'https://www.vpoxod.ru/route/nw/Vyborg_bay_kayak#content-top',
 'https://www.vpoxod.ru/route/karelia/Ladoga_kayak#content-top',
 'https://www.vpoxod.ru/route/nw/finnish_bay_kayak_island#content-top',
 'https://www.vpoxod.ru/route/karelia/Ladoga_kayak-9#content-top',
 'https://www.vpoxod.ru/route/nw/Lodoga_kayak_pvd#content-top',
 'https://www.vpoxod.ru/route/baikal/Baikalleto#content-top',
 'https://www.vpoxod.ru/route/baikal/BBL#content-top',
 'https://www.vpoxod.ru/route/altai/splav_po_reke_katun#content-top',
 'https://www.vpoxod.ru/route/kyrgyzstan/Iss

## Работаем с походом 

In [78]:
route_page = hike_list_urls[15]

In [112]:
class Route:
    def __init__(self, url):
        self.url = url
        self.page = session.get(self.url, headers=headers)
        self.soup = BeautifulSoup(self.page.text, 'html.parser')
        
        self.name = self.soup.find(attrs={'class':'main_top full_width'}).find().text
        self.main_route_info = self.soup.find(attrs={'class':'route_top_center'}).find_all('li')
        self.region = [col for col in self.main_route_info if 'Регион:' in col.text][0].text.replace('\n','')
        self.nitka = [col for col in self.main_route_info if 'Нитка:' in col.text][0].text.replace('\n','')
        self.distance = [col for col in self.main_route_info if 'Длина:' in col.text][0].text.replace('\n','')
        self.duration = [col for col in self.main_route_info if 'Длительность:' in col.text][0].text.replace('\n','')
        self.group_size = [col for col in self.main_route_info if 'Размер группы:' in col.text][0].text.replace('\n','')
        self.is_children = [col for col in self.main_route_info if 'Участие с детьми:' in col.text][0].text.replace('\n','')
        

In [111]:
main_info

[<li><div class="route-rating" itemscope="" itemtype="http://schema.org/Product"><div itemprop="aggregateRating" itemscope="" itemtype="http://schema.org/AggregateRating"><div class="rating-value"><a href="/route/kyrgyzstan/Issyk-kul/responses#tabs"><span class="rating-value-percent" style="width:100%"></span></a></div><meta content="7" itemprop="reviewCount"><meta content="5.0" itemprop="ratingValue"><a class="review-count" href="/route/kyrgyzstan/Issyk-kul/responses#tabs">7 отзывов</a> </meta></meta></div><meta content="Иссык-Куль: теплое море в ладонях Тянь-Шаня" itemprop="name"/></div></li>,
 <li><b>Регион:</b>
 Киргизия (<a href="/route/kyrgyzstan">все походы в этом регионе</a>)
 </li>,
 <li><b>Нитка:</b><span>Бишкек - село Орто-Орюктю - озеро Иссык-Куль - ущелье Джуку - ущелье Барскоон - каньон Сказка - село Бар-Булак - Бишкек</span></li>,
 <li><b>Длина:</b> 200 км + 20 км (На байдарках и пешком) </li>,
 <li><b><span class="text-dashed no-hover tippy" title='&lt;p&gt;Маршрут сред

In [113]:
route_object = Route(route_page)

In [82]:
route_object.soup

<!DOCTYPE html>
<html lang="ru"><head><meta content="_csrf" name="csrf-param"/><meta content="X-vCtJNaNw-KXwK71Q8pvtlIpa8HOYbRXyz008AzUVc3r_aHzB5mfbglRPPjVhvpnHzUmDAJtKEGaYaWr1oibw==" name="csrf-token"/><title>Тур по озеру Иссык-Куль на байдарках и его окрестностям, сроки походов в 2021 году</title><meta content="Комбинированная водно-пешая программа по северо-востоку Кыргызстана: совершим путешествие на байдарках по озеру Иссык-Куль в окружении гор, доедем до самых красивых каньонов и ущелий и совершим легкие треккинги. Присоединяйтесь!" name="description"/><meta charset="utf-8"/><meta content="width=device-width, initial-scale=1" name="viewport"/><meta content="telephone=no" name="format-detection"/><meta content="2asd0ffMNVewNSVSWXyB63QG1TfNk_JIWzM9GWV3r6Y" name="google-site-verification"/><meta content="Тур по озеру Иссык-Куль на байдарках и его окрестностям, сроки походов в 2021 году" property="og:title"/><meta content="vpoXod.ru" property="og:site_name"/><meta content="https://ww

In [114]:
route_object.url

'https://www.vpoxod.ru/route/kyrgyzstan/Issyk-kul#content-top'

In [119]:
route_object.is_children

'Участие с детьми: можно взять с собой детей от 14 лет '

In [105]:
main_info = route_object.soup.find(attrs={'class':'route_top_center'}).find_all('li')

In [109]:
[col for col in main_info if 'Регион:' in col.text][0].text.replace('\n','')

'Регион:Киргизия (все походы в этом регионе)'

In [110]:
main_info

[<li><div class="route-rating" itemscope="" itemtype="http://schema.org/Product"><div itemprop="aggregateRating" itemscope="" itemtype="http://schema.org/AggregateRating"><div class="rating-value"><a href="/route/kyrgyzstan/Issyk-kul/responses#tabs"><span class="rating-value-percent" style="width:100%"></span></a></div><meta content="7" itemprop="reviewCount"><meta content="5.0" itemprop="ratingValue"><a class="review-count" href="/route/kyrgyzstan/Issyk-kul/responses#tabs">7 отзывов</a> </meta></meta></div><meta content="Иссык-Куль: теплое море в ладонях Тянь-Шаня" itemprop="name"/></div></li>,
 <li><b>Регион:</b>
 Киргизия (<a href="/route/kyrgyzstan">все походы в этом регионе</a>)
 </li>,
 <li><b>Нитка:</b><span>Бишкек - село Орто-Орюктю - озеро Иссык-Куль - ущелье Джуку - ущелье Барскоон - каньон Сказка - село Бар-Булак - Бишкек</span></li>,
 <li><b>Длина:</b> 200 км + 20 км (На байдарках и пешком) </li>,
 <li><b><span class="text-dashed no-hover tippy" title='&lt;p&gt;Маршрут сред

In [60]:
hike_list[1].find()['href']

'/route/karelia/keret#content-top'

In [34]:
int(temp.find(attrs={'class':'active disabled'}).text)

25

In [30]:
for i in temp.children:
    print(i)

<li class="first"><a class="btn btn-outline btn-green" data-page="0" href="/route?per-page=48"></a></li>
<li class="prev"><a class="btn btn-outline btn-green" data-page="23" href="/route?per-page=48&amp;page=24"><i class="fa fa-angle-left fa-lg"></i> Назад</a></li>
<li><a class="btn btn-outline btn-green" data-page="20" href="/route?per-page=48&amp;page=21">21</a></li>
<li><a class="btn btn-outline btn-green" data-page="21" href="/route?per-page=48&amp;page=22">22</a></li>
<li><a class="btn btn-outline btn-green" data-page="22" href="/route?per-page=48&amp;page=23">23</a></li>
<li><a class="btn btn-outline btn-green" data-page="23" href="/route?per-page=48&amp;page=24">24</a></li>
<li class="active disabled"><span class="btn btn-outline btn-green">25</span></li>
<li class="next disabled"><span class="btn btn-outline btn-green">Вперед <i class="fa fa-angle-right fa-lg"></i></span></li>
<li class="last disabled"><span class="btn btn-outline btn-green"></span></li>


In [10]:

soup = BeautifulSoup(test_response.text, 'html.parser')

In [11]:
soup.find_all()

<!DOCTYPE html>
<html lang="ru"><head><meta content="_csrf" name="csrf-param"/><meta content="WW0ysn56HC1ulZnD04Sqb634JEFOc5EM0KinYpGBovExKQaBIT5NX1zv34vl3Zg46MxVdnlDo3yJ7dUn_ujRyQ==" name="csrf-token"/><title>Все маршруты</title><meta content="Маршруты походов по России и всему миру! Бурные реки, таинственные незабываемые горы и масса положительных эмоций - всё это ждет вас!" name="description"/><meta charset="utf-8"/><meta content="width=device-width, initial-scale=1" name="viewport"/><meta content="telephone=no" name="format-detection"/><meta content="2asd0ffMNVewNSVSWXyB63QG1TfNk_JIWzM9GWV3r6Y" name="google-site-verification"/><meta content="Все маршруты" property="og:title"/><meta content="vpoXod.ru" property="og:site_name"/><meta content="https://www.vpoxod.ru/route" property="og:url"/><meta content="ru_RU" property="og:locale"/><meta content="Маршруты походов по России и всему миру! Бурные реки, таинственные незабываемые горы и масса положительных эмоций - всё это ждет вас!" pro